In [ ]:
"""
Add the zebrafish jaw lib to python path
"""

import os, sys

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(""), "..", "..")))

In [ ]:
"""
The first thing we'll do is read a model from disk - we want to access its config file

"""

from fishjaw.model import model

jaw_model = model.load_model("attempt_3.pkl")
jaw_config = jaw_model.config

In [ ]:
"""
We'll fine tune on some quadrates that Wahab segmented - we'll want to read these from the RDSF and crop to the right region of interest

"""

import re
import pathlib

import tifffile
from tqdm import tqdm

wahab_labels = (
    jaw_config["rdsf_dir"]
    / pathlib.Path("1Felix and Rich make models/Training dataset Tiffs/Training set 1")
).glob("*.tif")

# Remove these ones, since the 3D tifs dont exist
bad_labels = re.compile(r"(351|401|420|441)")
wahab_labels = [label for label in wahab_labels if not bad_labels.search(label.name)]

# TODO remove - for now just use the first 2
wahab_labels = wahab_labels[:2]

# Read the labels
quadrate_labels = [tifffile.imread(path) for path in tqdm(wahab_labels)]
quadrate_labels = [(l == 4) | (l == 5)  for l in tqdm(quadrate_labels)]

In [ ]:
"""
Read in the images
"""

from fishjaw.util import files

img_paths = [files.get_3d_tif(label_path) for label_path in wahab_labels]
for p in img_paths:
    assert p.exists()

quadrate_imgs = [tifffile.imread(path) for path in tqdm(img_paths)]

In [ ]:
"""
Find the centre of the quadrates and crop the labels and images

"""

from scipy.ndimage import center_of_mass

centroids = [
    tuple(round(x) for x in center_of_mass(label)) for label in tqdm(quadrate_labels)
]

In [ ]:
from fishjaw.images import transform

window_size = transform.window_size(jaw_config)
cropped_labels = [transform.crop(l, c, window_size, centred=True) for l, c in zip(tqdm(quadrate_labels), centroids)]
cropped_quadrates = [transform.crop(i, c, window_size, centred=True) for i, c in zip(tqdm(quadrate_imgs), centroids)]

In [ ]:
centroids
from fishjaw.visualisation import images_3d

for img, label in zip(cropped_quadrates, cropped_labels):
    images_3d.plot_slices(img, label)

In [ ]:
"""
Plot the jaws and labels just to check

"""
from fishjaw